
<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header" style="float: left">
        <a class="navbar-brand" href="0_Forside.ipynb" target="_self"> <h2> &uarr; Tilbake til forsiden</h2></a>
    </div>
  </div>
</nav>

# Spill en melodi

Det er mye rart man kan gjøre med tallsekvenser. For eksempel er digitale lydopptak også i grunn ikke mer enn en sekvens med veldig hyppige målinger fra en mikrofon, noe som tilsier at man også kan syntetisere enkelte lyder matematisk med Python. For eksempel viser formelen nedenfor det matematiske uttrykket for en "ren sinustone" (slik som bl.a. typisk brukes i hørselstester) med frekvens lik $440$Hz (tonen A for musikkinteresserte).

$$x_{\text{tone\_A}}(t) = \sin(2\pi \cdot 440 \cdot t)$$

Koden nedenfor vil generere en slik "ren sinustone" med varignet en på 2 sekund. Variabelen `sample_rate` er nødvendig for å fortelle datamaskinen hvor raskt den skal "mate" nye tallverdier til høyttaleren, og betyr "antall målepunkter per sekund".

In [1]:
import numpy as np
from IPython.display import Audio

# Generer tonen
F = 440
tone_duration = 2.0 # Seconds
sample_rate = 8000 # Values per second in playback
t = np.linspace(0,  # Starttid t=0
                tone_duration, # Sluttid t = tone_duration
                int(sample_rate*tone_duration), # Antall datapunkt i tidsintervall
                endpoint=False) # Ekskluder sluttiden fra array
tone = np.sin(2*np.pi*F*t)

# Spill av lyden med funksjonen Audio
# (setter normalize=False og deler på 16 for å være litt skånsom med lydnivå)
Audio(tone/(max(abs(tone))*16), rate=sample_rate, normalize=False)

## a)

Lag en funksjon `generate_tone(frequency, duration, rate)` som kan brukes til å generere toner med valgfri frekvens `frequency` og lengde `duration`, med `rate` antall målepunkter per sekund.

In [2]:
### BEGIN SOLUTION
import numpy as np
def generate_tone(frequency, duration, rate):
    t = np.linspace(0, duration, int(rate*duration), endpoint=False)
    tone = np.sin(2*np.pi*frequency*t)
    return tone
### END SOLUTION

In [3]:
### BEGIN HIDDEN TESTS
from random import sample
from autofeedback import FunctionTests
import numpy as np

def tone_solution(frequency, duration, rate):
    t = np.linspace(0, duration, int(rate*duration), endpoint=False)
    tone = np.sin(2*np.pi*frequency*t)
    return tone
    
grading_results = FunctionTests(tone_solution)

try:
    grading_results.add_test_func(generate_tone)
except Exception as e:
    grading_results.log.append("Could not run tests, "+e.args[0])
else:
    N = 3  # Number of tests
    T_vals = sample([x/10 for x in range(1, 11)], N)
    F_vals = sample([x*100 for x in range(1, 11)], N)
    fs_vals = sample(list(range(4000, 13000, 1000)), N)
    for T, F, fs in zip(T_vals, F_vals, fs_vals):
        grading_results.test_return_value(F, T, fs)
round(grading_results.get_results()*4, 2)
### END HIDDEN TESTS

4.0

## b) 
Bruk funksjonen fra oppgave **a)** til å lage én lang array `melody` med mange toner etter hverandre. Hvilke toner, hvor langvarige og hvilken rekkefølge er oppgitt i tabellen under.

|Frekvens|Varighet|
|---|---|
|392Hz|0.9s|
|0Hz (stille)|0.1s|
|587Hz|0.9s|
|0Hz (stille)|0.1s|
|494Hz|0.4s|
|0Hz (stille)|0.1s|
|494Hz|0.4s|
|0Hz (stille)|0.1s|
|392Hz|0.9s|

Bruk 8000 målepunkt per sekund (`sample_rate = 8000`).\
*Hint: funksjonene `numpy.zeros()` og `numpy.concatenate()` kan være grei å se litt nærmere på.*<br>
*Hint 2: sicing av arrays kan også være nokså nyttig i denne oppgaven.*

In [4]:
### BEGIN SOLUTION
total_length = 0.9+0.1+0.9+0.1+0.4+0.1+0.4+0.1+0.9 # Seconds
melody = np.zeros(int(total_length*sample_rate))

melody[0:int(0.9*sample_rate)] = generate_tone(392, 0.9, sample_rate)
melody[int(1.0*sample_rate):int(1.9*sample_rate)] = generate_tone(587, 0.9, sample_rate)
melody[int(2.0*sample_rate):int(2.4*sample_rate)] = generate_tone(494, 0.4, sample_rate)
melody[int(2.5*sample_rate):int(2.9*sample_rate)] = generate_tone(494, 0.4, sample_rate)
melody[int(3.0*sample_rate):int(3.9*sample_rate)] = generate_tone(392, 0.9, sample_rate)
### END SOLUTION

Kjør cellen nedenfor for å lytte til melodien:

In [5]:
# Spill av lyden med funksjonen Audio
# (setter normalize=False og deler melodien på 16 for å være litt skånsom med lydnivå)
Audio(melody/(max(abs(melody))*16), rate=sample_rate, normalize=False)
### BEGIN HIDDEN TESTS
from autofeedback import CustomTests
import numpy as np

grading_result = CustomTests()

grading_result.test(sample_rate == 8000,
                    "Sample rate correct",
                    "Incorrect sample rate",
                    wgt=0.5)
grading_result.test(np.isclose(len(melody), sample_rate*3.9, atol=sample_rate*0.125),
                    f"Array 'melody' has length {len(melody)} which is correct",
                    f"Incorrect length for array 'melody'. Expected {sample_rate*3.9}, got {len(melody)}")

notes = [392, 587, 494, 494, 392]
durations = [0.9, 0.9, 0.4, 0.4, 0.9]
pauses = [0.1, 0.1, 0.1, 0.1, 0.0]
start = 0
melody /= np.max(np.abs(melody)) # Normalize


def get_loudest_tone(x, fs):
    Xf = np.abs(np.fft.rfft(x, n=fs))
    tone = np.argmax(Xf)
    return tone

for note, dur, pause in zip(notes, durations, pauses):


    stop = start + int(dur*sample_rate)
    melody_excerpt = melody[start:min(len(melody), stop)]
    tone = get_loudest_tone(melody_excerpt, sample_rate)
    if start < len(melody):
        grading_result.test(np.isclose(tone, note, atol=1),
                           f"{tone}Hz tone found in interval {start/sample_rate:.1f}s <= t <= {stop/sample_rate:.1f}s.",
                           f"{tone}Hz tone found in interval {start/sample_rate:.1f}s <= t <= {stop/sample_rate:.1f}s, expected {note}Hz.")
    else:
        grading_result.add_result(False, "interval {start/sample_rate:.1f}s <= t <= {stop/sample_rate:.1f}s out of bounds for array 'melody'.")
    start = stop + int(pause*sample_rate)
        
#grading_result.add_result(False, "test")
round(grading_result.get_results()*6, 2)
### END HIDDEN TESTS

6.0

<br>
<nav class="navbar navbar-default">
    <div class="container-fluid">
        <div class="navbar-header" style="float: left">
            <a class="navbar-brand" href="9_Plotting_3.ipynb" target="_self">&lt; Forrige side: <i>logaritmisk plotting</i></a>
            </div>
    </div>
</nav>